# Viculación: Segmentación de Estudiantes

 ## Names: Edgar Munoz Abad 

## Carga de datos y librerias

In [1]:
 ## Carga de datos: Luego de ordenar los datos en un solo archivo se procede 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
#import ace_tools as tools

In [2]:
path='data.xlsx'
df = pd.read_excel(path)


In [3]:
# mostrar la data
df.head()

,Institucion,CEDULA ESTUDIANTE,APELLIDOS Y NOMBRES,FECHA DE NACIMIENTO \nDÍA-MES-AÑO,TIENE CARNE \nSI/NO,TIPOD,PORCENTAJE\n%,GRADO O CURSO\n2023 2024
0,MANUELA CAÑISARES MATRIZ,2.250537e+09,MACIAS CAMPOVERDE YARELY FERNANDA,2019-07-02,SI,INTELECTUAL,0.36,INICIAL 2
1,MANUELA CAÑISARES MATRIZ,2.250537e+09,MACIAS CAMPOVERDE JANA VALENTINA,2019-07-02,SI,INTELECTUAL,0.36,INICIAL 2
2,MANUELA CAÑISARES MATRIZ,2.250538e+09,TAPUY GREFA ALEJANDRO ISAAC,2019-07-11,NO,NaN,NaN,INICIAL 2
3,MANUELA CAÑISARES MATRIZ,2.250466e+09,GOYES VILLALTA LUIS AARON,2017-12-05,SI,INTELECTUAL,0.36,1RO EGB
4,MANUELA CAÑISARES MATRIZ,1.352713e+09,SANCHEZ OLMEDO ALEXANDRA GUADALUPE,2017-10-06,SI,FISICA,0.85,1RO EGB


## Procesamiento

En esta seccion se procesara la data de cada una de las columnas. Al tener poca data no es faltible eliminar la data, si no q se aplicara tecnicas para autorellenar los valores faltante

### Transformacion de variables

In [4]:
df['TIPOD'].unique()

array(['INTELECTUAL', nan, 'FISICA', 'PSICOSOCIAL', 'VISUAL ',
       'AUDITIVA ', 'INTELECTUAL ', 'VISUAL', 'intelectual', 'visual',
       'Autismo'], dtype=object)

In [6]:

df.isnull().sum()

Institucion                           0
CEDULA ESTUDIANTE                    13
APELLIDOS Y NOMBRES                   0
FECHA DE NACIMIENTO \nDÍA-MES-AÑO    32
TIENE CARNE \nSI/NO                  28
TIPOD                                14
PORCENTAJE\n%                        23
GRADO O CURSO\n2023 2024              3
dtype: int64

In [9]:

#Esatndarizar solo en mayusculas
df['TIPOD'] = df['TIPOD'].str.upper()
# borrar espacio al inicio o final
df['TIPOD'] = df['TIPOD'].apply(lambda x: str(x).strip() if isinstance(x, str) else x)
df['APELLIDOS Y NOMBRES'] = df['APELLIDOS Y NOMBRES'].apply(lambda x: str(x).strip() if isinstance(x, str) else x)
#df['TIPOD'].unique()

In [ ]:
df['TIPOD'].unique()

### Procesar Nulos

In [8]:

df.isnull().sum()


Institucion                           0
CEDULA ESTUDIANTE                    13
APELLIDOS Y NOMBRES                   0
FECHA DE NACIMIENTO \nDÍA-MES-AÑO    32
TIENE CARNE \nSI/NO                  28
TIPOD                                14
PORCENTAJE\n%                        23
GRADO O CURSO\n2023 2024              3
dtype: int64

#### Nulos: Tiene Carnet, Tiene discapacidad, porcentaje, CEDULA

Se rellenara a las filas que tienen las 3 columnas nulas con la siguiente informacion: Tiene Carnet=NO, TipoD=NINGUNA, Porcentaje=0

In [10]:
#MOSTRAR LAS COLUMNAS DE 3 NULOS
df.loc[(df.loc[:, "TIENE CARNE \nSI/NO"].isnull()) & (df.loc[:, "TIPOD"].isnull()) & (df.loc[:, "PORCENTAJE\n%"].isnull())]

,Institucion,CEDULA ESTUDIANTE,APELLIDOS Y NOMBRES,FECHA DE NACIMIENTO \nDÍA-MES-AÑO,TIENE CARNE \nSI/NO,TIPOD,PORCENTAJE\n%,GRADO O CURSO\n2023 2024
6,MANUELA CAÑISARES MATRIZ,2.150546e+09,NAANCH TSENKUSH PAUL JON,NaT,NaN,NaN,NaN,1RO EGB
19,MANUELA CAÑISARES MATRIZ,2.250347e+09,ANDY SHIGUANGO NEYMAR ANGEL,NaT,NaN,NaN,NaN,4TO EGB
26,MANUELA CAÑISARES MATRIZ,2.250415e+09,QUEZADA ALARCON BRITHANY JOHANNA,NaT,NaN,NaN,NaN,NaN
38,MANUELA CAÑISARES MATRIZ,1.101205e+09,VASQUEZ ARDILA JONATHAN,NaT,NaN,NaN,NaN,5TO EGB
39,MANUELA CAÑISARES MATRIZ,2.250341e+09,TENEZACA REMACHE JENNIFER ADRIANA,NaT,NaN,NaN,NaN,5TO EGB
45,MANUELA CAÑISARES MATRIZ,2.200680e+09,MORA AGUINDA FLOR MARINA,NaT,NaN,NaN,NaN,6TO EGB
46,MANUELA CAÑISARES MATRIZ,2.200661e+09,GUALLCO TIPAN BRIGITH SAURI,NaT,NaN,NaN,NaN,6TO EGB
143,MANUELA CAÑISARES MATRIZ,2.200277e+09,QUIÑONEZ GREFA KEVIN JOEL,NaT,NaN,NaN,NaN,3ERO BACHILLERATO


In [11]:
# COLOCAR VALORES RESPECTIVOS PARA LOS 3 VALORES NULOS EN COMBINACION
def replace_values(row):
    if pd.isnull(row['TIENE CARNE \nSI/NO']) and  pd.isnull(row['TIPOD']) and  pd.isnull(row['PORCENTAJE\n%']):
        row['TIENE CARNE \nSI/NO'] = 'NO'
        row['TIPOD'] = 'NINGUNA'
        row['PORCENTAJE\n%'] = 0
    return row
df = df.apply(replace_values, axis=1)    

In [12]:
df.loc[(df.loc[:, "TIENE CARNE \nSI/NO"]=='NO') & (df.loc[:, "TIPOD"]=='NINGUNA') & (df.loc[:, "PORCENTAJE\n%"]==0)]

,Institucion,CEDULA ESTUDIANTE,APELLIDOS Y NOMBRES,FECHA DE NACIMIENTO \nDÍA-MES-AÑO,TIENE CARNE \nSI/NO,TIPOD,PORCENTAJE\n%,GRADO O CURSO\n2023 2024
6,MANUELA CAÑISARES MATRIZ,2.150546e+09,NAANCH TSENKUSH PAUL JON,NaT,NO,NINGUNA,0,1RO EGB
19,MANUELA CAÑISARES MATRIZ,2.250347e+09,ANDY SHIGUANGO NEYMAR ANGEL,NaT,NO,NINGUNA,0,4TO EGB
26,MANUELA CAÑISARES MATRIZ,2.250415e+09,QUEZADA ALARCON BRITHANY JOHANNA,NaT,NO,NINGUNA,0,NaN
38,MANUELA CAÑISARES MATRIZ,1.101205e+09,VASQUEZ ARDILA JONATHAN,NaT,NO,NINGUNA,0,5TO EGB
39,MANUELA CAÑISARES MATRIZ,2.250341e+09,TENEZACA REMACHE JENNIFER ADRIANA,NaT,NO,NINGUNA,0,5TO EGB
45,MANUELA CAÑISARES MATRIZ,2.200680e+09,MORA AGUINDA FLOR MARINA,NaT,NO,NINGUNA,0,6TO EGB
46,MANUELA CAÑISARES MATRIZ,2.200661e+09,GUALLCO TIPAN BRIGITH SAURI,NaT,NO,NINGUNA,0,6TO EGB
143,MANUELA CAÑISARES MATRIZ,2.200277e+09,QUIÑONEZ GREFA KEVIN JOEL,NaT,NO,NINGUNA,0,3ERO BACHILLERATO


#### Nulos: Tiene discapacidad

In [13]:

df.loc[(df.loc[:, "TIPOD"].isnull())]

,Institucion,CEDULA ESTUDIANTE,APELLIDOS Y NOMBRES,FECHA DE NACIMIENTO \nDÍA-MES-AÑO,TIENE CARNE \nSI/NO,TIPOD,PORCENTAJE\n%,GRADO O CURSO\n2023 2024
2,MANUELA CAÑISARES MATRIZ,2.250538e+09,TAPUY GREFA ALEJANDRO ISAAC,2019-07-11,NO,NaN,NaN,INICIAL 2
20,MANUELA CAÑISARES MATRIZ,2.250391e+09,ALVAREZ TROYA DANIEL FERNANDO,2015-09-05,NO,NaN,NaN,4TO EGB
31,MANUELA CAÑISARES MATRIZ,2.250422e+09,GARCIA GARCIA DARWIN ALEJANDRO,2012-12-09,NO,NaN,NaN,5TO EGB
47,MANUELA CAÑISARES MATRIZ,2.200661e+09,GREFA AGUINDA INTI RUMI,2012-08-22,SI,NaN,NaN,6TO EGB
49,MANUELA CAÑISARES MATRIZ,2.250138e+09,QUEZADA ZAMBRANO JOSE LUIS,2012-09-23,NO,NaN,NaN,6TO EGB
61,MANUELA CAÑISARES MATRIZ,2.250100e+09,LAPO JUELA SARAHI STEFANIA,2009-03-15,NO,NaN,NaN,8VO EGB


SE CARGA LOS VALORES EN FUNCION DE LA COLUMNA TIENE CARNE \nSI/NO

In [14]:
#RELLENAR LOS NULOS DE TIPO DE DISCAPACIDA CON NIGUNA A LOS QUE NO TIENE CARNET
df["TIPOD"] = np.where(df["TIENE CARNE \nSI/NO"]=='NO', 'NINGUNA',df["TIPOD"])

In [15]:

df.loc[(df.loc[:, "TIPOD"].isnull()) ]

,Institucion,CEDULA ESTUDIANTE,APELLIDOS Y NOMBRES,FECHA DE NACIMIENTO \nDÍA-MES-AÑO,TIENE CARNE \nSI/NO,TIPOD,PORCENTAJE\n%,GRADO O CURSO\n2023 2024
47,MANUELA CAÑISARES MATRIZ,2.200661e+09,GREFA AGUINDA INTI RUMI,2012-08-22,SI,NaN,NaN,6TO EGB


Rellenamos el valor que mas se repite

In [ ]:
df.dtypes



In [16]:
#RELLENAR LOS VALORES NULOS CON EL VALOR Q MAS SE REPITE

df['TIPOD'] = df['TIPOD'].fillna(df['TIPOD'].mode()[0])
df.loc[(df.loc[:, "APELLIDOS Y NOMBRES"]=='GREFA AGUINDA INTI RUMI') ]

,Institucion,CEDULA ESTUDIANTE,APELLIDOS Y NOMBRES,FECHA DE NACIMIENTO \nDÍA-MES-AÑO,TIENE CARNE \nSI/NO,TIPOD,PORCENTAJE\n%,GRADO O CURSO\n2023 2024
47,MANUELA CAÑISARES MATRIZ,2.200661e+09,GREFA AGUINDA INTI RUMI,2012-08-22,SI,INTELECTUAL,NaN,6TO EGB


In [17]:
df.isnull().sum()

Institucion                           0
CEDULA ESTUDIANTE                    13
APELLIDOS Y NOMBRES                   0
FECHA DE NACIMIENTO \nDÍA-MES-AÑO    32
TIENE CARNE \nSI/NO                  20
TIPOD                                 0
PORCENTAJE\n%                        15
GRADO O CURSO\n2023 2024              3
dtype: int64

In [18]:
df.loc[(df.loc[:, "TIENE CARNE \nSI/NO"].isnull())]

,Institucion,CEDULA ESTUDIANTE,APELLIDOS Y NOMBRES,FECHA DE NACIMIENTO \nDÍA-MES-AÑO,TIENE CARNE \nSI/NO,TIPOD,PORCENTAJE\n%,GRADO O CURSO\n2023 2024
167,MATRIZ JORGE RODRIGEZ,2.250024e+09,ANDRADE GOMEZ ANAHI THAIS,NaT,NaN,AUDITIVA,0.31,3ERO BACHILLERATO
168,MATRIZ JORGE RODRIGEZ,2.101127e+09,RIERA TOCTAGUNO ANGELICA KATHERINE,NaT,NaN,FISICA,0.44,2DO BACHILLERATO
169,MATRIZ JORGE RODRIGEZ,2.200496e+09,RAMIREZ ATACUSHI MARIA ISABEL,NaT,NaN,FISICA,0.82,2DO BACHILLERATO
170,MATRIZ JORGE RODRIGEZ,2.101127e+09,RIERA TOCTAGUNO ANGELICA KATHERINE,NaT,NaN,INTELECTUAL,0.44,2DO BACHILLERATO
171,MATRIZ JORGE RODRIGEZ,2.200354e+09,QUEVEDO ORTIZ JARET ALEXANDER,NaT,NaN,INTELECTUAL,0.65,2DO BACHILLERATO
172,MATRIZ JORGE RODRIGEZ,2.101127e+09,RIERA TOCTAGUNO ANGELICA KATHERINE,NaT,NaN,INTELECTUAL,0.44,2DO BACHILLERATO
173,MATRIZ JORGE RODRIGEZ,2.200215e+09,VITE TIXI JEFFERSON STEVEN,NaT,NaN,INTELECTUAL,0.35,3ERO BACHILLERATO
174,MATRIZ JORGE RODRIGEZ,2.101127e+09,RIERA TOCTAGUNO ANGELICA KATHERINE,NaT,NaN,FISICA,0.44,3ERO BACHILLERATO
175,MATRIZ JORGE RODRIGEZ,2.250041e+09,GRANDEZ AGILERA JOSE FERNANDO,NaT,NaN,AUTISMO,NaN,9NO EGB
176,FRANCISCO ORELLANA,NaN,GERMY GIOMAR ORDOÑEZ BRAVO,NaT,NaN,INTELECTUAL,0.25,3ERO BACHILLERATO


### Nulos: TIENE CARNE \nSI/NO

Se llena en funcion del tipo de discapacidad. Tiene discapacidad entonces tiene carnet.

In [21]:
df["TIENE CARNE \nSI/NO"] = np.where(df["TIPOD"]!='NINGUNA', 'SI',df["TIENE CARNE \nSI/NO"])

In [22]:
df.isnull().sum()

Institucion                           0
CEDULA ESTUDIANTE                    13
APELLIDOS Y NOMBRES                   0
FECHA DE NACIMIENTO \nDÍA-MES-AÑO    32
TIENE CARNE \nSI/NO                   0
TIPOD                                 0
PORCENTAJE\n%                        15
GRADO O CURSO\n2023 2024              3
dtype: int64

### Nulos: PORCENTAJE\n%

Se rellena en funcion de tipo de discapacida: Ninguna discapacidad entonce porcentaje 0

In [31]:
df.loc[(df.loc[:, "PORCENTAJE\n%"].isnull())]

,Institucion,CEDULA ESTUDIANTE,APELLIDOS Y NOMBRES,FECHA DE NACIMIENTO \nDÍA-MES-AÑO,TIENE CARNE \nSI/NO,TIPOD,PORCENTAJE\n%,GRADO O CURSO\n2023 2024
47,MANUELA CAÑISARES MATRIZ,2.200661e+09,GREFA AGUINDA INTI RUMI,2012-08-22,SI,INTELECTUAL,NaN,6TO EGB
115,MANUELA CAÑISARES MATRIZ,1.789576e+07,FORNEZ QUINTERO ANGEL GABRIEL,2007-05-30,SI,INTELECTUAL,NaN,1ERO BACHILLERATO
147,PRESIDENTE TAMAYO,2.351623e+09,ALLAUCA CAMPOS SOFIA PRISCILA,2018-06-23,SI,PSICOSOCIAL,NaN,1ERO EGB
148,PRESIDENTE TAMAYO,2.250514e+09,GUZMAN AGUINDA TONY JOSHUE,2018-12-15,SI,PSICOSOCIAL,NaN,1ERO EGB
149,PRESIDENTE TAMAYO,2.250334e+09,CEPEDA AGUALONGO ABIEL JHOSUA,2013-08-19,SI,PSICOSOCIAL,NaN,4TO EGB
150,PRESIDENTE TAMAYO,2.250237e+09,FARFAN MECIAS DYLAN RICARDO,2013-11-14,SI,PSICOSOCIAL,NaN,6TO EGB
151,PRESIDENTE TAMAYO,2.250234e+09,TACURI SOPLIN YADHIEL ALESSANDRO,2013-11-08,SI,PSICOSOCIAL,NaN,6TO EGB
152,PRESIDENTE TAMAYO,1.850989e+09,PALATE LIGÑA JOSE FERNANDO,2014-06-11,SI,VISUAL,NaN,5TO EGB
153,PRESIDENTE TAMAYO,1.756236e+09,JUNGAL DIAZ MAHU ARIANA,2008-10-19,SI,VISUAL,NaN,10MO EGB
175,MATRIZ JORGE RODRIGEZ,2.250041e+09,GRANDEZ AGILERA JOSE FERNANDO,NaT,SI,AUTISMO,NaN,9NO EGB


In [26]:
df["PORCENTAJE\n%"] = np.where(df["TIPOD"]=='NINGUNA', 0,df["PORCENTAJE\n%"])

In [28]:
df['PORCENTAJE\n%'].unique()

array([0.36, 0, 0.85, 0.84, 0.37, 0.77, 0.75, 0.79, 0.71, 0.88, 0.52,
       0.53, 0.5, 0.4, 0.76, 0.44, 0.8, 0.38, 0.41, 0.82, 0.46, 0.73, nan,
       0.9, 0.66, 0.91, 0.57, 0.49, 0.56, 0.32, 0.78, 0.67, 0.55, 0.45,
       0.7, 0.3, 0.58, 0.43, 0.63, 0.48, 0.6, 0.72, 0.86, 0.68, 0.95,
       0.65, 0.89, 0.42, 'S/N', 0.62, 0.54, 0.61, 0.33, 0.35, 0.81, 0.31,
       0.25, 0.59], dtype=object)

In [29]:
# se localiza un caratecterar no comun
df.loc[(df.loc[:, "PORCENTAJE\n%"]=='S/N')]

,Institucion,CEDULA ESTUDIANTE,APELLIDOS Y NOMBRES,FECHA DE NACIMIENTO \nDÍA-MES-AÑO,TIENE CARNE \nSI/NO,TIPOD,PORCENTAJE\n%,GRADO O CURSO\n2023 2024
115,MANUELA CAÑISARES MATRIZ,17895756.0,FORNEZ QUINTERO ANGEL GABRIEL,2007-05-30,SI,INTELECTUAL,S/N,1ERO BACHILLERATO


In [30]:
#cambaimos el S/N por NaN para rellenar la columan con el promedio
df["PORCENTAJE\n%"] = np.where(df["PORCENTAJE\n%"]=='S/N', np.nan ,df["PORCENTAJE\n%"])

In [32]:
# convertimos la variable porcentaje en nuemrico
df["PORCENTAJE\n%"] = pd.to_numeric(df["PORCENTAJE\n%"])


In [33]:

df.isnull().sum()


Institucion                           0
CEDULA ESTUDIANTE                    13
APELLIDOS Y NOMBRES                   0
FECHA DE NACIMIENTO \nDÍA-MES-AÑO    32
TIENE CARNE \nSI/NO                   0
TIPOD                                 0
PORCENTAJE\n%                        11
GRADO O CURSO\n2023 2024              3
dtype: int64

In [34]:
cursos=df['TIPOD'].unique()

In [36]:
#rellenamos el nulo con el promedio
# la funcion crea un filtro para cada conjunto de cursos en donde se tenga alguna discapacidad
for i in cursos:
    # Filtrar el DataFrame para para cada curso 
    mask = df["TIPOD"] == i
 # Calcular la media de las fechas disponibles para para el curso
    if df.loc[mask, 'PORCENTAJE\n%'].isna().all():
        continue  # Saltar si todas las entradas son NaN
    mean_date = df.loc[mask, 'PORCENTAJE\n%'].mean()
    #print(i)
    #print(mean_date)
    
    df.loc[mask, 'PORCENTAJE\n%'] = df.loc[mask, 'PORCENTAJE\n%'].fillna(mean_date) 



#df['PORCENTAJE\n%'] = df['PORCENTAJE\n%'].fillna(df['PORCENTAJE\n%'].mean())

#df.isnull().sum()



In [44]:
df.isnull().sum()

Institucion                           0
CEDULA ESTUDIANTE                    13
APELLIDOS Y NOMBRES                   0
FECHA DE NACIMIENTO \nDÍA-MES-AÑO    31
TIENE CARNE \nSI/NO                   0
TIPOD                                 0
PORCENTAJE\n%                         0
GRADO O CURSO\n2023 2024              3
dtype: int64

In [43]:
df.loc[(df.loc[:, "PORCENTAJE\n%"].isnull())]

,Institucion,CEDULA ESTUDIANTE,APELLIDOS Y NOMBRES,FECHA DE NACIMIENTO \nDÍA-MES-AÑO,TIENE CARNE \nSI/NO,TIPOD,PORCENTAJE\n%,GRADO O CURSO\n2023 2024


In [42]:
df = df.dropna(subset=['PORCENTAJE\n%'])

### Nulos: GRADO O CURSO\n2023 2024

In [ ]:
df.loc[(df.loc[:, "GRADO O CURSO\n2023 2024"].isnull())]

In [ ]:
# validamoos cuantos cursos existem

cursos=df['GRADO O CURSO\n2023 2024'].unique()
cursos

In [ ]:
# Contar cuántas filas tienen NaN  
num_nan = df['GRADO O CURSO\n2023 2024'].isnull().sum() 
# Generar valores aleatorios para las filas con NaN en 'Random_Category' 
np.random.seed(42)
random_choices = np.random.choice(cursos, num_nan, replace=True) 
# Asignar los valores aleatorios a las filas correspondientes 
random_choices

In [ ]:
df.loc[df['GRADO O CURSO\n2023 2024'].isnull(), 'GRADO O CURSO\n2023 2024'] = random_choices

In [ ]:
df.isnull().sum()


### Nulos: FECHA DE NACIMIENTO \nDÍA-MES-AÑO

In [ ]:
# mostramos fecha de nacimiento nulas

df.loc[(df.loc[:, "FECHA DE NACIMIENTO \nDÍA-MES-AÑO"].isnull())]

In [ ]:
df.dtypes

Para rellenar las fechas se lo va a realizar mediante un promedio de cada curso, para los estudiantes que tiene discapacidad.

In [ ]:
cursos=df['GRADO O CURSO\n2023 2024'].unique()
cursos


Se enecunentra mal escrito 1RO EGB===> 1ERO EGD 

In [ ]:
# corrregir sintaxis de los datos
filtro = (df["GRADO O CURSO\n2023 2024"] == "1RO EGB")
# Usar loc para actualizar el valor 
df.loc[filtro, 'GRADO O CURSO\n2023 2024'] = '1ERO EGB'

In [ ]:
cursos=df['GRADO O CURSO\n2023 2024'].unique()
cursos


Agrupar por cusros y sacar el promedio e insertar en los nulos los promedio de cada cursos para los alumnos con discapacida

In [ ]:
# la funcion crea un filtro para cada conjunto de cursos en donde se tenga alguna discapacidad
for i in cursos:
    # Filtrar el DataFrame para para cada curso 
    mask = (df["GRADO O CURSO\n2023 2024"] == i) & (df['TIPOD'] != 'NINGUNA') 
 # Calcular la media de las fechas disponibles para para el curso
    mean_date = df.loc[mask, 'FECHA DE NACIMIENTO \nDÍA-MES-AÑO'].mean()
    print(i)
    print(mean_date)
    
    df.loc[mask, 'FECHA DE NACIMIENTO \nDÍA-MES-AÑO'] = df.loc[mask, 'FECHA DE NACIMIENTO \nDÍA-MES-AÑO'].fillna(mean_date) 


In [ ]:
df.loc[(df["GRADO O CURSO\n2023 2024"] == "3ERO BACHILLERATO")]

In [ ]:
# validnado nulos
df.loc[(df.loc[:, "FECHA DE NACIMIENTO \nDÍA-MES-AÑO"].isnull())]

Se palica la misma logica para llenar la fecha de nacimiento para personas que no tienen discapacidad


In [ ]:
# la funcion crea un filtro para cada conjunto de cursos en donde se tenga alguna discapacidad
for i in cursos:
    # Filtrar el DataFrame para para cada curso 
    mask = (df["GRADO O CURSO\n2023 2024"] == i) & (df['TIPOD'] == 'NINGUNA') 
 # Calcular la media de las fechas disponibles para para el curso
    mean_date = df.loc[mask, 'FECHA DE NACIMIENTO \nDÍA-MES-AÑO'].mean()
   
        
    #print( mean_date)
    #print(mean_date)
    
    df.loc[mask, 'FECHA DE NACIMIENTO \nDÍA-MES-AÑO'] = df.loc[mask, 'FECHA DE NACIMIENTO \nDÍA-MES-AÑO'].fillna(mean_date) 


In [ ]:
# validnado nulos
df.loc[(df.loc[:, "FECHA DE NACIMIENTO \nDÍA-MES-AÑO"].isnull())]

Se inserta fechas normales en una poblacion sin discapacidad basandose en el curso que esta, tomando en cuenta que se ingresa a 1ero a los 5 años

In [ ]:
# Definir el filtro para las condiciones 
filtro1 = (df["GRADO O CURSO\n2023 2024"] == "3ERO BACHILLERATO") & (df['TIPOD'] == 'NINGUNA') 
filtro2 = (df["GRADO O CURSO\n2023 2024"] == "1ERO EGB") & (df['TIPOD'] == 'NINGUNA') 
# Usar loc para actualizar el valor 
df.loc[filtro1, 'FECHA DE NACIMIENTO \nDÍA-MES-AÑO'] = pd.to_datetime('2007-01-01')
df.loc[filtro2, 'FECHA DE NACIMIENTO \nDÍA-MES-AÑO'] = pd.to_datetime('2019-01-01')


In [ ]:

df.loc[(df["GRADO O CURSO\n2023 2024"] == "3ERO BACHILLERATO") & (df['TIPOD'] == 'NINGUNA')]

In [ ]:
df.isnull().sum()

## Calcular Edad

In [ ]:
#Se verifica el formato de la fecha
df['FECHA DE NACIMIENTO \nDÍA-MES-AÑO'] 

In [ ]:
# Funcion para calcular la edad hasta el 06-08-2024 en funcion de la fecha de nacimiento
from datetime import datetime
fecha_referencia = datetime(2024, 7, 2)
def calcular_edad(fecha_nacimiento, fecha_referencia):
    return fecha_referencia.year - fecha_nacimiento.year - ((fecha_referencia.month, fecha_referencia.day) < (fecha_nacimiento.month, fecha_nacimiento.day))

#df['Fecha de Nacimiento'] = pd.to_datetime(df['FECHA DE NACIMIENTO \nDÍA-MES-AÑO'])  # Asegúrate de que la columna sea de tipo datetime



In [ ]:
# Agregar una nueva columna de edad
df['Edad'] = df['FECHA DE NACIMIENTO \nDÍA-MES-AÑO'].apply(lambda x: calcular_edad(x, fecha_referencia))
df

## Seelccion de data para analisis

La cedula y apellidos y nombre seran eliminados debido a que son valores unicos en el caso de la cedula. se eliminara la fecha de nacimiento ya que esta relacionado con la edad

In [ ]:
df

In [ ]:
df_new=df.drop(columns=df.columns[[0,1,2]])
df_new

## Tipo de datos

In [ ]:

df_new.dtypes

In [ ]:
#categorical_mappings = {
 #   'TIENE CARNE \nSI/NO': 'category',
  #  'TIPOD': 'category',
   # 'GRADO O CURSO\n2023 2024': 'category'

#}

#df_new = df_new.astype(categorical_mappings)


In [ ]:
#df_new.dtypes

## Codificar valores Categoricas nominales

In [ ]:
df_new['TIPOD'].unique()


In [ ]:
# nominales 

carnet_mapping = {'SI': 1, 'NO': 0}
df_new['TIENE CARNE \nSI/NO'] = df_new['TIENE CARNE \nSI/NO'].map(carnet_mapping)
# colocamos en true para que borre una columna y asi evitar colinelaidad
df_new= pd.get_dummies(df_new, columns=['TIPOD'], prefix='TIPOD', drop_first=False)
#df = pd.get_dummies(df, columns=['Marital_Status'], prefix='Marital_Status', drop_first=True)


## Codificar valores categoricos ordinales

In [ ]:
df_new['GRADO O CURSO\n2023 2024'].unique()

In [ ]:
class_mapping = {'INICIAL 2': 0, '1ERO EGB': 1,
                '2DO EGB':2, '3ERO EGB':3, '4TO EGB':4,
       '5TO EGB':5, '6TO EGB':6, '7MO EGB':7, '8VO EGB':8, '9NO EGB':9, '10MO EGB':10,
       '1ERO BACHILLERATO':11, '2DO BACHILLERATO':12, '3ERO BACHILLERATO':13}

df_new['GRADO O CURSO\n2023 2024'] = df_new['GRADO O CURSO\n2023 2024'].map(class_mapping)


In [ ]:
df_new

## Correlacion

In [ ]:
cov_matrix = df_new.corr()
# Crear el mapa de calor
plt.figure(figsize=(15, 12))
sns.heatmap(cov_matrix, annot=True, fmt='.2f', cmap='coolwarm', cbar=True)
plt.title('Matriz de Correlacion')
plt.show(block=True)

Se eliminia la edad ya que tiene alta correlacion

In [ ]:

df_new=df_new.drop(['Edad','TIPOD_NINGUNA'], axis=1) 

In [ ]:
df_new.dtypes

In [ ]:
df_new['TIPOD_AUDITIVA'] = df_new['TIPOD_AUDITIVA'].astype(int)
df_new['TIPOD_AUTISMO'] = df_new['TIPOD_AUTISMO'].astype(int)
df_new['TIPOD_FISICA'] = df_new['TIPOD_FISICA'].astype(int)
df_new['TIPOD_INTELECTUAL'] = df_new['TIPOD_INTELECTUAL'].astype(int)
df_new['TIPOD_PSICOSOCIAL'] = df_new['TIPOD_PSICOSOCIAL'].astype(int)
df_new['TIPOD_VISUAL'] = df_new['TIPOD_VISUAL'].astype(int)

# Entrenamiento: Clustering

In [ ]:
df_new

## Normalizar

In [ ]:
# Normalizar los datos


scaler = StandardScaler()
df_scaled= scaler.fit_transform(df_new)


In [ ]:
df_scaled

In [ ]:
# Create an empty list
wcss=[]

# Create all possible cluster solutions with a loop
for i in range(1,30):
    # Cluster solution with i clusters
    kmeans = KMeans(i)
    # Fit the data
    kmeans.fit(df_scaled)
    # Find WCSS for the current iteration
    wcss_iter = kmeans.inertia_
    # Append the value to the WCSS list
    wcss.append(wcss_iter)

In [ ]:
wcss

In [ ]:
# Create a variable containing the numbers from 1 to 6, so we can use it as X axis of the future plot
number_clusters = range(1,30)
# Plot the number of clusters vs WCSS
plt.plot(number_clusters,wcss)
# Name your graph
plt.title('The Elbow Method')
# Name the x-axis
plt.xlabel('Number of clusters')
# Name the y-axis
plt.ylabel('Within-cluster Sum of Squares')

In [ ]:
# Fiddle with K (the number of clusters)
kmeans_new = KMeans(9)
# Fit the data
kmeans_new.fit(df_scaled)
# Create a new data frame with the predicted clusters
clusters_new = df_new.copy()
clusters_new['cluster'] = kmeans_new.fit_predict(df_scaled)

In [ ]:
clusters_new

In [ ]:

plt.scatter(clusters_new['PORCENTAJE\n%'], clusters_new['GRADO O CURSO\n2023 2024'], c=clusters_new['cluster'], cmap='rainbow')
plt.xlabel('PORCENTAJE\n%')
plt.ylabel('GRADO O CURSO\n2023 2024')
plt.title('Clustering con K-means')
plt.show()

In [ ]:
# Análisis descriptivo por clúster
cluster_analysis = clusters_new.groupby('cluster').mean()

In [ ]:
tools.display_dataframe_to_user(name="Cluster Analysis", dataframe=cluster_analysis)